<a href="https://colab.research.google.com/github/ferdinandrafols/IA_LLMs/blob/main/aula4_Finetune_decoder.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Aula 4 — Fine-tuning de um Transformer Decoder-only

Nesta prática faremos um **fine-tuning supervisionado (SFT)** em um **decoder-only causal LM** usando o menor modelo *útil* e popular para demos no Hugging Face:

- **Modelo base:** `distilbert/distilgpt2`

Passos:
1. Carregar/definir documentos (toy ou pasta local)
2. Tokenização + *grouping* em blocos (CLM)
3. Fine-tuning com `Trainer`
4. Inferência

In [ ]:
# (Opcional) Instalar dependências no ambiente local/Colab
!pip -q install transformers datasets accelerate

In [ ]:
import os
import torch
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    DataCollatorForLanguageModeling,
    TrainingArguments,
    Trainer
)

print("torch:", torch.__version__)
print("cuda available:", torch.cuda.is_available())
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch: 2.9.0+cpu
cuda available: False


## 1) Dataset — Documentos (toy) ou pasta local

### Opção A (didática): documentos toy
### Opção B (real): ler `.txt` de uma pasta local

> Para aula, a Opção A roda rápido e mostra o pipeline completo.


In [ ]:
# Opção A: documentos toy
docs = [
    "UFU forma estudantes de IA aplicada para muitas áreas.",
    "Um transformer decoder-only prevê o próximo token com auto-regressão.",
    "Fine-tuning ajusta um modelo pré-treinado para um domínio específico.",
    "Temperatura controla aleatoriedade; top-k e top-p controlam o corte do vocabulário.",
    "A Universidade Federal de Uberlândia é top demais.",
    "A Universidade Federal de Uberlândia é top demais mesmo."
]

ds = Dataset.from_dict({"text": docs}).train_test_split(test_size=0.1, seed=42)
ds

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 5
    })
    test: Dataset({
        features: ['text'],
        num_rows: 1
    })
})

In [ ]:
from datasets import Dataset
from glob import glob
# The user provided a specific file to use as the dataset.
# We will read this single file into our documents.
file_path = "/content/500 Frases Mais Famosas do Obama (1).txt"

try:
    with open(file_path, "r", encoding="utf-8") as f:
        # Split the content by lines to create multiple documents
        docs = [line.strip() for line in f.readlines() if line.strip()]

    if not docs:
        raise ValueError("No content found in the file or all lines were empty.")

    # Adjust test_size if there are very few documents
    test_size = 0.1
    if len(docs) < 2:
        # If only one document, use it for both train and test (or skip split if not feasible)
        print("Warning: Only one document found. Skipping train_test_split.")
        ds = Dataset.from_dict({"text": docs})
    else:
        ds = Dataset.from_dict({"text": docs}).train_test_split(test_size=test_size, seed=42)

    print("Dataset loaded from:", file_path)
    print(ds)

except FileNotFoundError:
    print(f"Error: The file {file_path} was not found. Please ensure the file exists.")
    docs = []
    ds = Dataset.from_dict({"text": []})
except Exception as e:
    print(f"An error occurred while reading the file: {e}")
    docs = []
    ds = Dataset.from_dict({"text": []})

Dataset loaded from: /content/500 Frases Mais Famosas do Obama (1).txt
DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 270
    })
    test: Dataset({
        features: ['text'],
        num_rows: 30
    })
})


## 2) Tokenizer + Modelo base

Usaremos o tokenizer do modelo base e definiremos `pad_token`, pois GPT-2 não tem por padrão.


In [ ]:
model_id = "distilbert/distilgpt2"

tok = AutoTokenizer.from_pretrained(model_id)

# GPT-2 não define pad_token por padrão; necessário para batches com padding
if tok.pad_token is None:
    tok.pad_token = tok.eos_token

model = AutoModelForCausalLM.from_pretrained(model_id).to(device)

print("Vocab size:", len(tok))
print("Model loaded:", model_id)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/76 [00:00<?, ?it/s]

GPT2LMHeadModel LOAD REPORT from: distilbert/distilgpt2
Key                                        | Status     |  | 
-------------------------------------------+------------+--+-
transformer.h.{0, 1, 2, 3, 4, 5}.attn.bias | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Vocab size: 50257
Model loaded: distilbert/distilgpt2


## 3) Tokenização + agrupamento em blocos (Causal LM)

Em *Causal Language Modeling*, a label é o próprio `input_ids` deslocado internamente pelo modelo.
Uma forma padrão é concatenar tudo e quebrar em blocos fixos (`block_size`).


In [ ]:
block_size = 64  # pequeno para rodar rápido

def tokenize_fn(batch):
    return tok(batch["text"], truncation=True)

tok_ds = ds.map(tokenize_fn, batched=False, remove_columns=["text"])

def group_texts(examples):
    concatenated = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated["input_ids"])
    print('total_len = ', total_length)
    total_length = (total_length // block_size) * block_size
    print('total_len = ', total_length)
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

lm_ds = tok_ds.map(group_texts, batched=True)
collator = DataCollatorForLanguageModeling(tokenizer=tok, mlm=False)

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

Map:   0%|          | 0/270 [00:00<?, ? examples/s]

total_len =  3063
total_len =  3008


Map:   0%|          | 0/30 [00:00<?, ? examples/s]

total_len =  348
total_len =  320


## 4) Fine-tuning (SFT) com `Trainer`

> Com o seguinte código, abstraímos a parte do treino e executar 10 épocas para o modelo aprender o conteúdo do novo corpus.


In [ ]:
args = TrainingArguments(
    output_dir="./ft_tiny_gpt2",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    learning_rate=5e-4,
    logging_steps=1,
    eval_strategy="epoch",
    save_strategy="no",
    report_to="none"
)

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=lm_ds["train"],
    eval_dataset=lm_ds["test"],
    data_collator=collator
)

trainer.train()

/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
`loss_type=None` was set in the config but it is unrecognized. Using the default loss: `ForCausalLMLoss`.


Epoch,Training Loss,Validation Loss
1,3.020652,2.920737
2,1.646516,3.291842
3,1.260301,3.431842
4,1.116676,3.542227
5,0.791227,3.943805
6,0.584562,4.408114
7,0.439681,4.672380
8,0.245149,4.823079
9,0.281424,5.294633
10,0.172406,5.388661


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:668: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


TrainOutput(global_step=240, training_loss=0.9855389434223374, metrics={'train_runtime': 416.098, 'train_samples_per_second': 1.13, 'train_steps_per_second': 0.577, 'total_flos': 7675592048640.0, 'train_loss': 0.9855389434223374, 'epoch': 10.0})

## 5) Inferência depos do finetune



In [ ]:
prompt=tok("UFU ", return_tensors = "pt", truncation = True).input_ids[0,]
inputs = prompt.unsqueeze(0).to(device)
with torch.no_grad():
    out_ids = model.generate(
        inputs,
        max_new_tokens=10,
        do_sample=True,
        pad_token_id=tok.eos_token_id,
        temperature= 0.7,
        top_k = 50,
        top_p = 1.0
    )
print(f"Inference: {tok.decode(out_ids[0])}")

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Inference: UFU Ã’s future.We must strengthen the


In [ ]:
print("\n--- Inferência em Documentos de Treino ---")
train_docs = ds['train']['text']

for i, doc_text in enumerate(train_docs):
    initial_tokens = tok(doc_text, return_tensors="pt", truncation=True).input_ids[0, :10]
    inputs = initial_tokens.unsqueeze(0).to(device) # unsqueeze to add batch dimension

    with torch.no_grad():
        out_ids = model.generate(
            inputs,
            max_new_tokens=10,
            do_sample=True,
            pad_token_id=tok.eos_token_id,
            temperature= 0.7,
            top_k = 50,
            top_p = 1.0
        )

    # Decode the sequence
    prompt_text = tok.decode(initial_tokens, skip_special_tokens=True)
    generated_text = tok.decode(out_ids[0], skip_special_tokens=True)
    print(f"doc{i}: {prompt_text}*{generated_text[len(prompt_text):]}*")



--- Inferência em Documentos de Treino ---
doc0: We measure our success by the opportunities we create for* all.We owe it to future generations to act*
doc1: We owe it to future generations to act boldly and* wisely.We must create a future worthy of our*
doc2: We must open doors of opportunity for every child.*We must ensure access to healthcare for every citizen.*
doc3: We must ensure that our justice system is fair and* impartial.We must ensure that every American has the*
doc4: We must help communities rebuild and recover after hardship.*We must ensure that our children inherit a nation worthy*
doc5: We must remember that America has always been a nation* of strivers.We must support the innovators*
doc6: We must ensure that our children inherit a nation worthy* of their dreams.We will respond to the threats*
doc7: We will respond to the threats of climate change knowing* that failure to do so would undermine our prosperity.*
doc8: We must reject rhetoric that seeks to divide us.*We

# Exercício prático

Escolha um prompt que não estava nos dados de treino, mas que seja do mesmo domínio. Faça comparações. Documente a saída do modelo base (sem treino) e do modelo ajustado. Verifique se o modelo "aprendeu" termos técnicos ou o vocabulário específico que você forneceu nos arquivos .txt.